# Tornando um texto em uma base de dados para o Text-Fabric

Como tornar um texto qualquer em uma base de dados que possa ser utilizado pelo Text-Fabric? Primeiro, é necessário o texto em um formato legível pelo programa. Ele pode ser colocado manualmente ou a partir de um arquivo de texto existente. O ideial é um texto no formato .txt que se concentra apenas nos caracteres, sem qualquer outro código oculto dentro dele.

O arquivo "caminhoacristo.txt" contém os primeiros parágrafos do livro _Caminho a Cristo_ de Ellen White.

* `#` marca o início de um novo capítulo;

* `$` marca um metadado. "author" é uma chave. Ele não faz parte do texto em si.

* `##` marca o início de um parágrafo;

* `###` marca o início de uma citação bíblica;

* _linhas em branco_ separam os parágrafos.



In [1]:
# As próximas linhas de código carregam bibliotecas, classes e módulos necessários para o procedimento.

# "os" é uma biblioteca de interfaces de sistemas operacionais. (https://docs.python.org/3/library/os.html)
import os 

# "re" é uma biblioteca de operadores de expressões. (https://docs.python.org/3/library/re.html)
import re 

# classe do TF que permite carregar e salvar dados. (https://annotation.github.io/text-fabric/tf/fabric.html)
from tf.fabric import Fabric 

# classe para converter um conjunto de dados em uma base do TF (https://annotation.github.io/text-fabric/tf/convert/walker.html)
from tf.convert.walker import CV 

O próximo passo é chamar o TF e permitir que ele veja o diretório onde o resultado deve ser colocado.

In [2]:
# TF_DIR é o objeto que será criado para conter o endereço do diretório onde o resultado do procedimento será alocado.

BASE = os.path.expanduser('~')
ORG = 'tf_notebooks_Clacir'
REPO = 'Walker Procedure'
RELATIVE = 'tf'

TF_DIR = os.path.expanduser(f'{BASE}/{ORG}/{REPO}/{RELATIVE}')

VERSION = '0.2'

TF_PATH = f'{TF_DIR}/{VERSION}'
TF = Fabric(locations=TF_PATH, silent=True)

# Configuração

O ambiente de dados do TF é um conjunto de arquivos `.tf` que iniciam com alguns poucos metadados. Estes metadados contêm os valores de um recurso para cada nó ou limite de um gráfico.

##  O tipo _slot_

Uma das principais características do TF é sua granularidade. Para isso ele usa um tipo de dado chamado _slot_. Ele põe se tornar qualquer coisa: uma palavra, os morfemas dessa palavra, uma série de caracteres. Isso dependerá dos interesses do programador/pesquisador.

In [4]:
slotType = 'word'

## Origem

Ao deparar com dados do TF na internet, os usuários precisam ter alguma noção da origem e dos tipos de dados que serão manipulados. Os metadados especificados aqui serão usados em todos os recursos do TF gerados.

In [6]:
generic = {
    'name' : 'Textos de EGW',
    'compiler' : 'Clacir Virmes Junior',
    'source' : 'Ellen White Writings',
    'url' : 'https://egwwritings.org/',
    'version' : '0.0.1',
    'purpuse' : 'teste',
    'status': 'em andamento'  
}

## Texto

Algumas características da apresentação do texto podem ser especificadas no recurso `otext`. Esse é um recurso sem dados; ele apenas possui metadados. Ele contém especificiações para cada seção estrutural do seu texto e os formatos do texto.

### Estrutura da Seção

O TF assume que há no seu texto duas ou três seções com as quais ele pode trabalhar. Para cada seção você tem de especificar o tipo de nó e o recurso que contém o nome e o número da seção (`sectionTypes` e `sectionFeatures`).

### Estrutura Própria

Eventualmente você também pode definir estruturas de seção mais extensivas e flexíveis para seus próprios propósitos (`sectionTypes` e `sectionFeatures`). Os dois sistemas podem usar os mesmos tipos e recursos, mas são completamente independentes.

### Formatos de Texto

Quando você pede ao TF renderizar os nós do tipo _slot_ (ou seja, nós com texto), o TF precisa saber que recursos renderizar.
O formato do texto é um modelo com marcadores de posição para os recursos que você quer usar.

In [ ]:
otext = {
    #fmt é uma biblioteca para formatação de texto
    
    #tudo o que vier entre chaves {} são variáveis definidas mais à frente
    
    #define que o texto completo é composto de letras e pontuação, variáveis que serão definidas a seguir
    'fmt:text-orig-full': '{letters}{punc}',
    
    #define que o final de uma linha se dá no último caracter da linha. 
    'fmt:line-term': 'line#{terminator} ',
    
    #define que a linha padrão são um conjunto de letras e o sinal de terminação
    'fmt:line-default': '{letters:XXX}{terminator}',
    
    #define os tipos de seção
    'sectionTypes': 'book,chapter,sentence',
    
    'sectionFeatures': 'title,number,number',
    
    #define a estrutura do texto
    'structureTypes': 'book,chapter,sentence,line',
    
    'structureFeatures': 'title,number,number,number',
}

### Tipos

Normalmente os valores são conjuntos de caracteres. Mas se se sabe que haverá valores numéricos, eles podem ser declarados como números.

In [2]:
intFeatures = {
  'number',
  'gap',
}

### Descritores

Você pode descrever o que cada variável contém. É possível usar tantas variáveis quanto necessárias

Uma boa _descrição_ é muito útil, especialmente porque elas se tornam um auxílio durante a programação.

In [3]:
featureMeta = {
    'number': {
        'description': 'number of chapter, or sentence in chapter, or line in sentence',
    },
    'gap': {
        'description': '1 for words that occur between [ ], which are inserted by the editor',
    },
    'title': {
        'description': 'the title of a book',
    },
    'author': {
        'description': 'the author of a book',
    },
    'terminator': {
        'description': 'the last character of a line',
    },
    'letters': {
        'description': 'the letters of a word',
    },
    'punc': {
        'description': 'the punctuation after a word',
    },
}

# Diretor

Esse é o centro do procedimento. É a função que você vai escrever, codificar.
Essa função precisa passar pelo material-fonte e disparar ações no TF. O TF faz o trabalho através dessas ações e você se concentra nos aspectos dos seus dados.

Cada vez que o diretor encontra um novo objeto textual na fonte, ele solicita uma ação requerendo um novo nó. O diretor pega a receita, através da qual ele pode solicitar ações subsequentes para aquele nó, como adicionar valores a ele.

Quando o objeto está feito, o diretor solicita a ação de término, `terminate`.

Durante tudo isso, a máquina `cv` está ocupada traduzindo essas ações na construção de um gráfo próprio do TF representando todo o material-fonte que você expôs a ele.

Note que:

* Se você quiser encerrar um nó você não precisa se preocupar se ele existe ou já foi terminado: é só encerrar;

* Se você encerrar um nó você pode retomá-lo mais tarde;

* Quando você adiciona nós, slots ou não-slots, há uma mágina nos bastidores:

    - quando um nó **slot** é adicionado, ele será ligado a todos os nós não-slots ativos, isto é, aqueles que você encerrou ou não retomou;

    - Quando um **non-slot** é adicionado, ele se torna automaticamente ativo, no sentido de que ele será ligado aos nós slot subsequentes, antes de ser encerrado ou depois de ser retomado;

* Se um erro fatal for encontrado, o diretor pode simplesmente dizer `cv.stop(message)`: o diretor é responsável por retornar o controle depois de solicitar `cv.stop`;

* Se as ações envolvem nós de seções, eles serão checados se todos os slots ocorrerem numa seção e se grandes seções como livros não começarem, terminarem ou terminarem denrto de pequenas seções dentro, como versos. Avisos serão dados, mas você pode suprimi-los;

* Depois que a função 'diretor' terminar, o TF fará várias checagens sobre o resultado.

In [ ]:
# define a função 'director' como um procedimento de conversão 'cv'
def director(cv):
  
  # dict é uma coleção ordenada, mutável e que não permite duplicatas.
  counter = dict(
    sentence=0,
    line=0,
  )
  cur = dict(
    book=None,
    chapter=None,
    sentence=None,
  )

  # wordRe é uma variável de recompilação que estabelece seus possíveis valores, isto é, letras e números
  wordRe = re.compile(r'^(.*?)([^A-Za-z0-9]*)$')
  # metaRe é uma variável de recompilação  
  metaRe = re.compile(r'^\$\s*([^= ]+)\s*=\s*(.*)')
  
  for line in source.strip().split('\n'):
    line = line.rstrip()
    if not line:
      cv.terminate(cur['sentence'])               # action
      for ntp in counter:
        counter[ntp] += 1
      cur['sentence'] = cv.node('sentence')       # action
      cv.feature(
        cur['sentence'],
        number=counter['sentence'],
      )                                           # action
      continue
      
    if line.startswith('# '):
      for ntp in ('sentence', 'chapter', 'book'):
        cv.terminate(cur[ntp])                    # action
        cur[ntp] = None         
      title = line[2:].strip()
      cur['book'] = cv.node('book')               # action
      for ntp in counter:
        counter[ntp] = 0
      cv.feature(
        cur['book'],
        title=title,
      )                                           # action
      continue

    if line.startswith('## '):
      for ntp in ('sentence', 'chapter'):
        cv.terminate(cur[ntp])                    # action
        cur[ntp] = None         
      number = line[2:].strip()
      cur['chapter'] = cv.node('chapter')         # action
      for ntp in counter:
        counter[ntp] = 0
      cv.feature(
        cur['chapter'],
        number=number,
      )                                           # action
      continue

    if line.startswith('$'):
      match = metaRe.match(line)
      if not match:
        cv.stop(
          f'Malformed metadata line: "{line}"'
        )                                         # action
        return
      name = match.group(1)
      value = match.group(2)
      cv.feature(
        cur['book'],
        **{name: value},
      )                                           # action
      continue
        
    if not cur['sentence']:
      cur['sentence'] = cv.node('sentence')       # action
      counter['sentence'] += 1
      cv.feature(
        cur['sentence'],
        number=counter['sentence'],
      )                                           # action
      
    cur['line'] = cv.node('line')                 # action
    counter['line'] += 1
    cv.feature(
      cur['line'],
      terminator=line[-1],
      number=counter['line'],
    )                                             # action
    
    gap = False
    for word in line.split():
      if word.startswith('['):
        gap = True
        cv.terminate(cur['line'])                 # action
        w = cv.slot()                             # action
        cv.feature(w, gap=1)                      # action
        word = word[1:]
      elif word.endswith(']'):
        w = cv.slot()                             # action
        cv.resume(cur['line'])                    # action
        cv.feature(w, gap=1)                      # action
        gap = False
        word = word[0:-1]
      else:
        w = cv.slot()
        if gap:
          cv.feature(w, gap=1)                    # action

      (letters, punc) = wordRe.findall(word)[0]
      cv.feature(w, letters=letters)              # action
      if punc:
        cv.feature(w, punc=punc)                  # action
    cv.terminate(cur['line'])                     # action
    curLine = None
    
  # just for informational purposes
  print('\nINFORMATION:', cv.activeTypes(), '\n') # action
  
  for ntp in ('sentence', 'chapter', 'book'):
    cv.terminate(cur[ntp])                        # action

  cv.meta(
    'punc', remark='a bit more info is needed',
  )                                               # action